# Circuitos do 2º Grau com Falhas — Modelo SMT

## Objetivo
O objetivo deste trabalho é modelar e analisar **circuitos booleanos do 2º grau com falhas**,
de acordo com a formulação apresentada nos apontamentos.  
Pretende-se representar as falhas apenas nas *gates* AND, construir o modelo lógico correspondente,
e estudar a forma como essas falhas afetam o resultado e a estimativa do segredo `z`.

---

## Problema
Um circuito do 2º grau é descrito pelo polinómio booleano:
\[
f(p; x) = o \oplus \langle a \cdot x \rangle \oplus (\langle b \cdot x \rangle \times \langle c \cdot x \rangle)
\]
em que os vetores \( a, b, c \in \{0,1\}^n \) e o *offset* \( o \in \{0,1\} \)
são gerados pseudo-aleatoriamente a partir de uma “master seed” \( s \in \{0,1\}^\kappa \).  
O segredo \( z \) é uma raiz do polinómio sem falhas, i.e. \( f(p; z) = 0 \).

Neste contexto, é necessário:
1. Construir o circuito e o modelo SMT que represente falhas nas *gates* AND.  
2. Determinar, para uma saída observada com falhas, possíveis combinações de \( z' \) e falhas que a expliquem.  
3. Avaliar a probabilidade máxima de falhas que não alteram o *output* esperado \( 0^n \).

---

## Solução Implementada
A solução foi desenvolvida em **Python**, com recurso ao **NumPy** e ao **Z3 Solver**.  
As principais componentes são:

- **Geração de parâmetros** (`a, b, c, o`) a partir da *seed* `s` via PRNG.  
- **Avaliação do polinómio do 2º grau** para um dado vetor `x`.  
- **Modelo SMT com falhas AND** representadas por variáveis booleanas, permitindo falhas *stuck-at-0*.  
- **Redundância tripla (TMR)**: cada AND é replicada 3 vezes e combinada através de uma *majority gate*, garantindo tolerância a falhas únicas.  
- **Procura de explicações** (`z'`, falhas) que reproduzem um *output* observado usando o solver SMT.  
- **Estimativa da probabilidade máxima de falhas** sem alteração do *output*.

In [ ]:
!pip install z3-solver

from typing import List, Tuple
import numpy as np
from z3 import Bool, Solver, And, Or, Xor, sat


# Função para gerar os parâmetros do circuito

def generate_parameters(n: int, kappa: int, s: List[int]) -> Tuple[List[int], List[int], List[int], int]:
    # Converte a lista de bits 's' em um número inteiro para usar como seed
    seed_int = int("".join(map(str, s)), 2)
    rng = np.random.default_rng(seed_int)  # Inicializa RNG determinístico

    # Gera três vetores binários aleatórios (0 ou 1) de tamanho n
    a = rng.integers(0, 2, n).tolist()
    b = rng.integers(0, 2, n).tolist()
    c = rng.integers(0, 2, n).tolist()

    # Gera um bit de offset aleatório
    o = int(rng.integers(0, 2))
    return a, b, c, o


# Função que calcula a saída do polinômio binário

def poly2(o: int, a: List[int], b: List[int], c: List[int], x: List[int]) -> int:
    # Produto escalar binário mod 2 (AND e soma mod 2)
    ax = sum(ai & xi for ai, xi in zip(a, x)) % 2
    bx = sum(bi & xi for bi, xi in zip(b, x)) % 2
    cx = sum(ci & xi for ci, xi in zip(c, x)) % 2

    # Saída final = offset XOR ax XOR (bx AND cx)
    return o ^ ax ^ (bx & cx)


# Função para a porta MAJ (majority)

def maj_gate(x1, x2, x3):
    # Retorna 1 se pelo menos dois dos três inputs forem 1
    return Or(And(x1, x2), And(x1, x3), And(x2, x3))


# Constrói modelo SMT do circuito com TMR (Triple Modular Redundancy)

def build_smt_model_TMR(a: List[int], b: List[int], c: List[int], o: int, n: int):
    # Função auxiliar para XOR de uma lista de variáveis
    def xor_list(vars):
        if len(vars) == 0: return False
        if len(vars) == 1: return vars[0]
        return Xor(*vars)

    # Cria variáveis booleanas x_i (entradas) e y_i (saídas)
    x = [Bool(f"x_{i}") for i in range(n)]
    y = [Bool(f"y_{i}") for i in range(n)]

    # Cria variáveis auxiliares para as portas AND do TMR
    d_ands = [[Bool(f"d_and_{i}_{j}") for j in range(3)] for i in range(n)]

    s = Solver()  # Inicializa solver SMT

    # Para cada bit de saída
    for i in range(n):
        # Calcula XOR das entradas selecionadas por a, b e c
        a_term = xor_list([x[j] for j in range(n) if a[j]==1])
        b_term = xor_list([x[j] for j in range(n) if b[j]==1])
        c_term = xor_list([x[j] for j in range(n) if c[j]==1])

        # Constrói 3 ANDs replicados (TMR)
        and_gates = [Or(d_ands[i][j], And(b_term, c_term)) for j in range(3)]
        maj_out = maj_gate(and_gates[0], and_gates[1], and_gates[2])  # Majority

        # Saída y_i = offset XOR a_term XOR saída do majority dos ANDs
        y_expr = Xor(bool(o), a_term, maj_out)
        s.add(y[i] == y_expr)

    return s, x, y, d_ands


# Função para encontrar a primeira solução SMT que satisfaça o output observado

def find_first_solution_smt_TMR(y_obs: List[int], n: int, kappa: int, s_seed: List[int]):
    # Gera parâmetros do circuito
    a, b, c, o = generate_parameters(n, kappa, s_seed)

    # Constrói o modelo SMT
    solver, x_vars, y_vars, d_vars = build_smt_model_TMR(a, b, c, o, n)

    # Força o modelo a produzir o output observado
    for i in range(n):
        solver.add(y_vars[i] == bool(y_obs[i]))

    # Resolve o SMT e retorna primeira solução encontrada
    if solver.check() == sat:
        m = solver.model()
        z_found = [1 if m[xv] else 0 for xv in x_vars]  # Entradas estimadas
        # Lista de ANDs que estão "falhando"
        faults = [(i,j) for i in range(n) for j in range(3) if m[d_vars[i][j]]]
        return z_found, faults
    else:
        return None, None


# Função para determinar quais falhas podem ocorrer sem alterar output

def maximal_faults_preserving_output(z: List[int], s: List[int], n: int, kappa: int, prob_threshold=None):
    a, b, c, o = generate_parameters(n, kappa, s)
    fz = poly2(o, a, b, c, z)  # Saída do polinômio

    # Identifica bits críticos (AND b[i]&c[i]&z[i] = 1)
    critical = [i for i in range(n) if (b[i] & c[i] & z[i])==1]
    safe = [i for i in range(n) if i not in critical]  # Bits que podem falhar

    if prob_threshold is None:
        return {'safe_to_fail': safe, 'cannot_fail': critical, 'output': fz}

    # Calcula probabilidade máxima de falha que preserva output
    k = len(critical)
    eps_max = 1.0 - prob_threshold**(1.0/(3*k)) if k>0 else 1.0
    return {'epsilon_max': eps_max, 'k_critical': k, 'output': fz}

# Demonstração / Teste das funções
if __name__ == "__main__":
    n = 5
    kappa = 3
    z = [1,0,1,1,0]
    s = [1,0,1]

    # Gera parâmetros do circuito
    a, b, c, o = generate_parameters(n, kappa, s)
    print("a =", a)
    print("b =", b)
    print("c =", c)
    print("offset =", o)

    # Calcula saída esperada sem falhas
    y_expected = poly2(o, a, b, c, z)
    print(f"\nOutput esperado (sem falhas): {y_expected}")

    # Cria vetor de observações replicando saída esperada
    y_obs = [y_expected]*n
    z_est, faults_est = find_first_solution_smt_TMR(y_obs, n, kappa, s)
    print("\nEstimativa z' e falhas AND (primeira solução):")
    print("z' =", z_est)
    print("falhas =", faults_est)

    # Determina falhas máximas que preservam saída
    info = maximal_faults_preserving_output(z, s, n, kappa, prob_threshold=0.99)
    print("\nMaximização da probabilidade de falhas AND sem alterar output:")
    print(info)


a = [1, 1, 0, 1, 0]
b = [1, 1, 0, 1, 0]
c = [0, 0, 1, 0, 0]
offset = 0

Output esperado (sem falhas): 0

Estimativa z' e falhas AND (primeira solução):
z' = [0, 0, 0, 0, 0]
falhas = []

Maximização da probabilidade de falhas AND sem alterar output:
{'epsilon_max': 1.0, 'k_critical': 0, 'output': 0}


# Exercício 2 — Modelagem FOTS + Bounded Model Checking (BMC)

## Objetivo
Modelar o algoritmo de multiplicação de inteiros positivos representados em vetores de bits como um **FOTS (Finite Observable Transition System)** e verificar propriedades usando **Bounded Model Checking (BMC)**.

Especificamente, pretende-se:
1. Construir o modelo de transição usando `BitVec`s de tamanho `n` para representar `a`, `b`, `x`, `y` e `z`.
2. Identificar e codificar:
   - Estado inicial
   - Relação de transição
   - Estado de erro (overflow)
3. Verificar se a propriedade:
   \[
   x \cdot y + z = a \cdot b
   \]
   é um **invariante** do sistema.
4. Restringindo o estado inicial por:
   \[
   N \le a,b \le M,
   \]
   verificar se o **estado de erro é inatingível** em até `N` passos (BMC).

---

## Problema
O programa analisado implementa multiplicação usando o método de acumulação sucessiva:

1. Inicialmente `x = a`, `y = b`, `z = 0`.
2. Enquanto `y > 0`:
   - Se `y` é par: faz-se deslocamento `x = x << 1`, `y = y >> 1`.
   - Se `y` é ímpar: faz-se `z = z + x`, `y = y - 1`.
3. Se qualquer deslocamento ou soma provocar **overflow**, o programa entra num **estado de erro**.
4. O objetivo é provar se esse erro pode ocorrer, e se a relação entre os valores internos (`x`, `y`, `z`) e os valores iniciais (`a`, `b`) permanece correta durante toda a execução.

---

## Solução
O que fiz foi modelar o algoritmo como um sistema de transição com variáveis em BitVec, o que permite capturar o overflow. Depois desenrolei o sistema até K passos usando Bounded Model Checking. Verifiquei se a propriedade x*y+z = a*b era um invariante, e o solver encontrou um contra-exemplo devido a overflow. Ao restringir os valores iniciais de a e b para um intervalo, verifiquei que o estado de erro deixa de ser atingível, o que mostra segurança para esse intervalo.

In [ ]:
# mult_bmc.py
# Requer: z3-solver (pip install z3-solver)
from z3 import *

def build_bmc(n, K, check_bounds=False, N=None, M=None, debug=False):
    """
    Constrói o modelo FOTS (Finite Observable Transition System) (sistema de transição) desenrolado em K passos.
    - n = número de bits (largura dos BitVecs)
    - K = profundidade do unrolling (Bounded Model Checking)
    - check_bounds = se True, adiciona no estado inicial N ≤ a,b ≤ M
    """
    
    # Função auxiliar para criar variáveis BitVec com nome e passo k
    bv = lambda name, k: BitVec(f"{name}_{k}", n)

    # pc_k é o program counter em cada passo (estado de controlo)
    pc = [ Int(f"pc_{k}") for k in range(K+1) ]

    # a e b são os operandos iniciais (só existem no passo 0 e propagam-se)
    a = bv("a", 0)
    b = bv("b", 0)

    # x, y, z são os valores internos do algoritmo, com cópias para todos os passos
    x = [ bv("x", k) for k in range(K+1) ]
    y = [ bv("y", k) for k in range(K+1) ]
    z = [ bv("z", k) for k in range(K+1) ]

    s = Solver()

    # Restringe o domínio de pc a valores válidos do autómato
    for k in range(K+1):
        s.add(Or([pc[k] == v for v in [0,1,2,3,4,5]]))

    # Estado inicial (passo 0)
    s.add(pc[0] == 0)          # Começa no estado 0 (start)
    s.add(x[0] == a)           # x = a
    s.add(y[0] == b)           # y = b
    s.add(z[0] == BitVecVal(0, n))  # z = 0

    # Se quisermos impor limites N ≤ a,b ≤ M no estado inicial:
    if check_bounds:
        assert N is not None and M is not None
        s.add(UGE(a, BitVecVal(N, n)))   # a >= N (unsigned)
        s.add(ULE(a, BitVecVal(M, n)))   # a <= M (unsigned)
        s.add(UGE(b, BitVecVal(N, n)))
        s.add(ULE(b, BitVecVal(M, n)))

    # Função auxiliar para detectar overflow no shift (se bit mais significativo=1 antes do shift)
    def shl_overflow(xbv):
        return Extract(n-1, n-1, xbv) == BitVecVal(1, 1)

    # Função auxiliar para detectar overflow na soma (soma de z+x for menor que z)
    def add_overflow(zbv, xbv, resbv):
        return ULT(resbv, zbv)   # res < z → overflow

    # Definição da relação de transição k → k+1
    # y==0 → pc=5 (termina)
    # y par → x=x<<1, y=y>>1
    # y ímpar → z=z+x, y=y-1
    # Se overflow → pc=4 (erro)
    for k in range(K):

        # Estado inicial → entra na loop (pc = 1)
        s.add(Implies(pc[k] == 0,
                      And(pc[k+1] == 1,
                          x[k+1] == x[k],
                          y[k+1] == y[k],
                          z[k+1] == z[k])))

        # Se y == 0 → terminar (pc = 5)
        s.add(Implies(And(pc[k] == 1, y[k] == BitVecVal(0,n)),
                      And(pc[k+1] == 5,
                          x[k+1] == x[k], y[k+1] == y[k], z[k+1] == z[k])))

        # Caso y seja par → shift: x = x << 1, y = y >> 1
        cond_even = And(pc[k] == 1, y[k] != BitVecVal(0,n),
                        Extract(0,0,y[k]) == BitVecVal(0,1))

        # Resultado do shift (BitVec)
        res_x_shl = (x[k] << 1)
        y_shr = LShR(y[k], 1)

        # Se overflow no shift → erro (pc = 4)
        s.add(Implies(And(cond_even, shl_overflow(x[k])),
                      pc[k+1] == 4))

        # Caso contrário aplica transição normal
        s.add(Implies(And(cond_even, Not(shl_overflow(x[k]))),
                      And(pc[k+1] == 1,
                          x[k+1] == res_x_shl,
                          y[k+1] == y_shr,
                          z[k+1] == z[k])))

        # Caso y seja ímpar → acumulação: z = z + x, y = y - 1
        cond_odd = And(pc[k] == 1, y[k] != BitVecVal(0,n),
                       Extract(0,0,y[k]) == BitVecVal(1,1))

        res_add = z[k] + x[k]

        # Se soma deu overflow → erro
        s.add(Implies(And(cond_odd, add_overflow(z[k], x[k], res_add)),
                      pc[k+1] == 4))

        # Caso normal sem overflow
        s.add(Implies(And(cond_odd, Not(add_overflow(z[k], x[k], res_add))),
                      And(pc[k+1] == 1,
                          x[k+1] == x[k],
                          y[k+1] == y[k] - BitVecVal(1,n),
                          z[k+1] == res_add)))

        # Se já está no erro → permanece no erro
        s.add(Implies(pc[k] == 4,
                      And(pc[k+1] == 4,
                          x[k+1] == x[k], y[k+1] == y[k], z[k+1] == z[k])))

        # Se terminou (pc = 5) → permanece terminado
        s.add(Implies(pc[k] == 5,
                      And(pc[k+1] == 5,
                          x[k+1] == x[k], y[k+1] == y[k], z[k+1] == z[k])))

    return s, (a,b,x,y,z,pc)


def check_invariant(n, K):
    """
    Verifica se a propriedade x*y + z = a*b é um INVARIANTE até K passos.
    """
    s, (a,b,x,y,z,pc) = build_bmc(n, K, check_bounds=False)
    #pega no modelo criado e pergunta ao solver se a propriedade falha em algum passo , se existe algum k onde xy+z != ab
    violation = []
    for k in range(K+1):
        # Promovemos as variaveis de n bits para 2n bits para evitar overflow artificial
        x2 = ZeroExt(n, x[k])
        y2 = ZeroExt(n, y[k])
        z2 = ZeroExt(n, z[k])
        a2 = ZeroExt(n, a)
        b2 = ZeroExt(n, b)
        lhs = x2 * y2 + z2   # valor interno
        rhs = a2 * b2        # produto real
        violation.append(lhs != rhs)  # condição de quebra da propriedade

    s.push()
    s.add(Or(*violation))   # Pergunta: existe algum k onde a propriedade falha?

    res = s.check()
    if res == sat:
        m = s.model()
        # Encontrar o primeiro passo que quebra a propriedade
        for k in range(K+1):
            xk = m.eval(x[k]).as_long()
            yk = m.eval(y[k]).as_long()
            zk = m.eval(z[k]).as_long()
            a_v = m.eval(a).as_long()
            b_v = m.eval(b).as_long()
            if xk * yk + zk != a_v * b_v:
                s.pop()
                return False, k, m
    #retorna contra-exemplo se encontrar violação
    s.pop()
    return True, None, None


def check_safety_with_bounds(n, K, N, M):
    """
    Verifica se o estado de erro (pc = 4) é alcançável
    quando N ≤ a,b ≤ M.
    """
    s, (a,b,x,y,z,pc) = build_bmc(n, K, check_bounds=True, N=N, M=M)
    # pergunto ao solver se o estado de erro 4 é alcançável em algum passo
    reach_error = [pc[k] == 4 for k in range(K+1)]
    s.add(Or(*reach_error))

    res = s.check()
    if res == sat:
        return False, s.model()   # inseguro
    else:
        return True, None          # seguro


if __name__ == "__main__":
    n = 8
    K = 20
    print("Checking invariant up to K =", K)
    ok, k, model = check_invariant(n, K)
    if ok:
        print("Nenhuma violação da propriedade encontrada.")
    else:
        print("Invariante falhou no passo", k)
        print(model)

    N, M = 0, 10
    print("\nChecking safety with bounds N={}, M={} up to K={}".format(N,M,K))
    safe, model2 = check_safety_with_bounds(n, K, N, M)
    if safe:
        print("Sem estados de erro dentro dos limites.")
    else:
        print("Erro alcançável:")
        print(model2)


Checking invariant up to K = 20
Invariant violated at step 7
[a_0 = 32,
 pc_5 = 1,
 pc_4 = 1,
 y_7 = 92,
 pc_16 = 4,
 z_1 = 0,
 x_15 = 240,
 pc_1 = 1,
 pc_17 = 4,
 z_7 = 191,
 z_9 = 191,
 x_17 = 240,
 pc_10 = 4,
 x_0 = 32,
 z_17 = 191,
 x_19 = 240,
 x_14 = 240,
 y_0 = 135,
 x_1 = 32,
 z_3 = 32,
 x_6 = 128,
 y_18 = 92,
 pc_13 = 4,
 pc_8 = 4,
 y_6 = 32,
 z_20 = 191,
 x_16 = 240,
 x_2 = 32,
 pc_14 = 4,
 z_11 = 191,
 x_11 = 240,
 x_10 = 240,
 z_12 = 191,
 b_0 = 135,
 z_8 = 191,
 pc_15 = 4,
 y_17 = 92,
 z_4 = 96,
 x_18 = 240,
 y_1 = 135,
 y_20 = 92,
 pc_7 = 4,
 pc_11 = 4,
 x_3 = 64,
 y_8 = 92,
 pc_19 = 4,
 pc_3 = 1,
 x_20 = 240,
 x_12 = 240,
 y_9 = 92,
 z_14 = 191,
 y_11 = 92,
 x_9 = 240,
 z_15 = 191,
 y_5 = 33,
 pc_18 = 4,
 z_2 = 32,
 y_12 = 92,
 y_13 = 92,
 pc_12 = 4,
 z_10 = 191,
 z_19 = 191,
 x_4 = 64,
 z_16 = 191,
 z_0 = 0,
 y_14 = 92,
 z_13 = 191,
 z_18 = 191,
 pc_6 = 1,
 z_5 = 96,
 x_5 = 128,
 z_6 = 224,
 y_10 = 92,
 y_16 = 92,
 pc_0 = 0,
 y_4 = 66,
 x_7 = 240,
 y_15 = 92,
 pc_2 = 1,